## Packages to Import

In [52]:
import numpy as np
import skimage as sk
from skimage.io import imread, imsave
from glob import glob
import napari
from cellpose import io, models, train 
import os
from tqdm import tqdm

## Randomly split the images and labels into test and training datasets
Images were previously split by Z and sorted to only keep the slices that had labels

In [53]:
X = sorted(glob('../Microglia_CellPose_Model/NonEmpty_Slices/*.tif'))
X = list(map(imread,X))
Y = sorted(glob('../Microglia_CellPose_Model/NonEmpty_Masks/*.tif'))
Y = list(map(imread,Y))
rng = np.random.RandomState(42)
ind = rng.permutation(len(X))
n_val = max(1, int(round(0.15 * len(ind))))
ind_train, ind_val = ind[:-n_val], ind[-n_val:]
X_val, Y_val = [X[i] for i in ind_val]  , [Y[i] for i in ind_val]
X_trn, Y_trn = [X[i] for i in ind_train], [Y[i] for i in ind_train]

Set save locations for train and test sets, then save imgs and labels with matching IDs.

In [54]:
save_train = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Train/'
save_test = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Test/'


CellPose is VERY picky about the structure of the images. Be sure to 

In [55]:
for i in tqdm(range(len(Y_trn))):
    squeezed_Y = np.squeeze(Y_trn[i])
    squeezed_X = np.squeeze(X_trn[i])
    imsave(os.path.join(save_train,'0'+str(i)+'_masks.tif'),squeezed_Y,check_contrast=False)
    imsave(os.path.join(save_train,'0'+str(i)+'_img.tif'),squeezed_X,check_contrast=False)

for i in tqdm(range(len(Y_val))):
    squeezed_Y = np.squeeze(Y_val[i])
    squeezed_X = np.squeeze(X_val[i])
    imsave(os.path.join(save_test,'0'+str(i)+'_masks.tif'),squeezed_Y,check_contrast=False)
    imsave(os.path.join(save_test,'0'+str(i)+'_img.tif'),squeezed_X,check_contrast=False)

100%|██████████| 22/22 [00:00<00:00, 24.64it/s]


## Using Cellpose to add training to the Cyto3 model.

In [ ]:
train_dir = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Train/'
test_dir = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Test/'
io.logger_setup()

output = io.load_train_test_data(train_dir, test_dir, image_filter="_img",
                                mask_filter="_masks", look_one_level_down=False)
images, labels, image_names, test_images, test_labels, image_names_test = output

# e.g. retrain a Cellpose model
model = models.CellposeModel(model_type="cyto3")

model_path, train_losses, test_losses = train.train_seg(model.net,
                            train_data=images, train_labels=labels,
                            channels=[0,0], normalize=True,
                            test_data=test_images, test_labels=test_labels,
                            weight_decay=1e-4, SGD=True, learning_rate=0.1,
                            n_epochs=100, model_name="my_new_model")

In [56]:
img = imread("E:/ParadaKusz_Lab/TIF_Files/Control_MO/Control_MO_04.tif")

In [57]:
io.logger_setup()
model = models.CellposeModel(gpu=True,model_type="Microglia_LargeZ_V2") # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'


creating new log file
2025-06-26 12:14:33,215 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-06-26 12:14:33,216 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.7.0+cu118
2025-06-26 12:14:33,218 [INFO] >> Microglia_LargeZ_V2 << model set to be used
2025-06-26 12:14:33,220 [INFO] ** TORCH CUDA version installed and working. **
2025-06-26 12:14:33,221 [INFO] >>>> using GPU (CUDA)
2025-06-26 12:14:33,354 [INFO] >>>> loading model C:\Users\kristin.gallik\.cellpose\models\Microglia_LargeZ_V2
2025-06-26 12:14:33,581 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-06-26 12:14:33,583 [INFO] >>>> model diam_labels =  43.143 (mean diameter of training ROIs)


In [58]:
channels =[[0,0]]
masks, _, _ = model.eval(img, channels=channels, z_axis=0,stitch_threshold=0.05, flow_threshold=0.4,cellprob_threshold=0.0)

2025-06-26 12:15:09,061 [INFO] multi-stack tiff read in as having 33 planes 1 channels
2025-06-26 12:15:27,970 [INFO] 100%|##########| 33/33 [00:17<00:00,  1.91it/s]
2025-06-26 12:15:29,511 [INFO] network run in 18.79s
2025-06-26 12:15:29,513 [INFO] 0%|          | 0/33 [00:00<?, ?it/s]
2025-06-26 12:15:29,525 [INFO] No cell pixels found.
2025-06-26 12:15:49,283 [INFO] 100%|##########| 33/33 [00:19<00:00,  1.67it/s]
2025-06-26 12:15:49,284 [INFO] stitching 33 planes using stitch_threshold=0.050 to make 3D masks


100%|██████████| 32/32 [00:00<00:00, 35.70it/s]


2025-06-26 12:15:51,964 [INFO] masks created in 22.45s


In [59]:
viewer = napari.view_image(img, name='img', scale =[4.5,0.3,0.3])
viewer.add_labels(masks, name='masks',scale=[4.5,0.3,0.3])

2025-06-26 12:16:07,359 [WARNING] DirectWrite: CreateFontFaceFromHDC() failed (Indicates an error in an input file such as a font file.) for QFontDef(Family="", pointsize=12, pixelsize=16, styleHint=5, weight=50, stretch=100, hintingPreference=0) LOGFONT("MS Sans Serif", lfWidth=0, lfHeight=-16) dpi=96


<Labels layer 'masks' at 0x224606436d0>